In [1]:
import numpy as np
import os
print("Current Working Directory " , os.getcwd())

# import nltk
# def ExtractNP(text):
#     nounphrases = []
#     words = nltk.word_tokenize(text)
#     tagged = nltk.pos_tag(words)

#     grammar = r"""
#         NP:
#             {<JJ*><NN+><IN><NN>}
#             {<NN.*|JJ>*<NN.*>}
#         """
#     chunkParser = nltk.RegexpParser(grammar)
#     tree = chunkParser.parse(tagged)
#     for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):
#         myPhrase = ''
#         for item in subtree.leaves():
#             myPhrase += ' ' + item[0]
#         nounphrases.append(myPhrase.strip())
#         # print(myPhrase)
#     nounphrases = list(filter(lambda x: len(x.split()) > 1, nounphrases))
#     return nounphrases


# from sklearn.feature_extraction.text import TfidfVectorizer
# test = TfidfVectorizer(dat1[0:10])
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy as np 

def IDF(corpus, unique_words):
    idf_dict={}
    N=len(corpus)
    for i in unique_words:
        count=0
        for sen in corpus:
            if i in sen.split():
                count=count+1
            idf_dict[i]=(math.log((1+N)/(count+1)))+1
    return idf_dict 

def fit(whole_data):
    unique_words = set()
    if isinstance(whole_data, (list,)):
        for x in whole_data:
            for y in x.split():
                if len(y)<2:
                    continue
                unique_words.add(y)
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}
        Idf_values_of_all_unique_words=IDF(whole_data,unique_words)
    return vocab, Idf_values_of_all_unique_words

def transform(dataset,vocabulary,idf_values):
     sparse_matrix= csr_matrix((len(dataset), len(vocabulary)), dtype=np.float64)
     for row  in range(0,len(dataset)):
       number_of_words_in_sentence=Counter(dataset[row].split())
       for word in dataset[row].split():
           if word in  list(vocabulary.keys()):
               tf_idf_value=(number_of_words_in_sentence[word]/len(dataset[row].split()))*(idf_values[word])
               sparse_matrix[row,vocabulary[word]]=tf_idf_value
    #  print("NORM FORM\n",normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False))
     output = normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False)
     return output


import TeKET
from TeKET import Tree
from TeKET import Node
# from TeKET import Root
from TeKET import TreeManager
from TeKET import ExtractCandidate
from TeKET import NounTFCalculation

# import TeKET_comments
# from TeKET_comments import Tree
# from TeKET_comments import Node
# # from TeKET import Root
# from TeKET_comments import TreeManager
# from TeKET_comments import ExtractCandidate
# from TeKET_comments import NounTFCalculation

# import TFIDF_train_data
# from TFIDF_train_data import TFIDF
# from TFIDF_train_data import PrecisionRecallF1Calculation
# from TFIDF_train_data import DataProcessing
# import TFIDF_test_data


def find_index(word, phrase):
    i = 0
    for w in phrase:
        if w == word:
            return(i)
            break
        i = i + 1


Current Working Directory  /Users/hyoinan/OneDrive - The Ohio State University/[2019-2024 OSU]/3-2022-Spring/STAT7620/Project


In [2]:
file1 = open("task05-TRAIN/train/H-37.txt", "r")
count = 0
dat1 = np.array([])
 
while True:
    line = file1.readline()
    if not line:
        break
    dat1 = np.append(dat1, line.strip())

my_str = ""
for line in dat1:
    my_str = my_str + " " + str(line)
# my_str = my_str.split(". ")
# dat1 = np.array(my_str)

# candidate = np.array([])
# for sen in dat1:
#     candidate_phrase = ExtractNP(sen)
#     candidate = np.append(candidate, candidate_phrase)

candidate = ExtractCandidate(my_str)
candidate_phrases = candidate.extract_candidate_chunks()
# print(candidate.extract_candidate_chunks())
candidate = candidate.CleaningCandidatePhrases(candidate_phrases) # To make the phrase of the basic form

Vocabulary, idf_of_vocabulary = fit(candidate) 

final_output = transform(candidate, Vocabulary, idf_of_vocabulary) # TF-IDF values

/Users/hyoinan/opt/anaconda3/envs/pytorch_env/lib/python3.8/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


## Get Similar phrases

In [3]:
def preparation (root, final_output, candidate):

    similar_candidate = []
    sen_ind = []
    sen_number = 0
    for sen in candidate:
        if sen.split().count(root)>0:
            similar_candidate.append(sen)
            sen_ind.append(sen_number)
        sen_number = sen_number + 1
    # the first element will be used to construct a tree, and the rest of it will be used to grow/prune the tree

    # # Solution for (1) & (2): removing dulicate words & the non-alphabetic words from each sentence in similar candidate phrases
    # similar_candidate_clean = []
    # for sen in similar_candidate:
    #     words = sen.split() # [( ), ( ), ..., ( )] 
    #     sen = " ".join(sorted(set(words), key=words.index)) # removing dulicate words
    #     sen_clean = []
    #     for word in sen.split():
    #         if (word.isalpha() or word.isspace()) == True:
    #             sen_clean.append(word)
    #     sen_clean = " ".join(str(e) for e in sen_clean)
    #     similar_candidate_clean.append(sen_clean)
    # similar_candidate = similar_candidate_clean

    # Solution for (1): removing dulicate words
    similar_candidate_clean = []
    for sen in similar_candidate:
        words = sen.split() # [( ), ( ), ..., ( )] 
        sen = " ".join(sorted(set(words), key=words.index)) # removing dulicate words
        similar_candidate_clean.append(sen)
    similar_candidate = similar_candidate_clean

    # Find TF-IDF values for the similar candidate
    index_matrix = np.array(final_output.nonzero())
    similar_TFIDF = []
    for row_ind in sen_ind:
        ind = (index_matrix[0,] == row_ind) # find the corresponding phrase
        col_ind = index_matrix[1, ind]
        tf_idf_k = np.asarray(final_output[row_ind, col_ind].todense()).flatten()
        similar_TFIDF.append(tf_idf_k)

    # Solution for (2) & (3): Removing the problematic sentences for now
    remove_list_ind = []
    for l in range(len(similar_candidate)):
        sim_sen = similar_candidate[l].split()
        logic = (len(sim_sen) == len(similar_TFIDF[l]))
        if logic == False:
            # print('[phrase ' + str(l) + '] number of words: ' + str(len(similar_candidate[l].split())) + ', number of TFIDFs: ' + str(len(similar_TFIDF[l])))
            remove_list_ind.append(l)
    remove_list_ind.sort(reverse=True, key=int)
    for i in remove_list_ind:
        del similar_candidate[int(i)]
        del similar_TFIDF[int(i)]

    # print(similar_TFIDF)
    # print(similar_candidate[0].split())
    sigma1_index = similar_candidate[0].split().index(root)
    root_tfidf = similar_TFIDF[0][sigma1_index]

    return similar_candidate, similar_TFIDF, root_tfidf

## Get root words

In [4]:
TF = NounTFCalculation()
TF = TF.NounTF(candidate)
# print(TF) # possible roots?

sorted_TF = sorted(TF, key=lambda tf: tf[1], reverse=True)
root_word_list = [x[0] for x in sorted_TF]
print(root_word_list)

['svm', 'spam', 'onlin', 'data', 'set', 'perform', 'result', 'exampl', 'detect', 'method', 'valu', 'email', 'blog', 'cost', 'rosvm', 'vector', 'number', 'margin', 'comput', 'featur', 'splog', 'time', 'high', 'section', 'machin', 'task', 'comment', 'size', 'test', 'optim', 'word', 'iter', 'figur', 'filter', 'train', 'smo', 'text', 'updat', 'hyperplan', 'experi', 'relax', 'problem', 'content-bas', 'effect', 'hypothesi', 'proceed', 'support', 'classif', 'paramet', 'benchmark', 'note', 'messag', 'confer', 'previou', 'trec', 'comparison', 'use', 'larg', 'content', 'linear', 'maximum', 'space', 'same', 'seendata', 'trec06p']


In [5]:
# TF = NounTFCalculation()

# sorted_TF = sorted(TF, key=lambda tf: tf[1], reverse=True)
# #print(sorted_TF)

# root_list = [x[0] for x in sorted_TF]

# # Remove the root words that contain non-alphabetic components 
# new_root_list = []
# for root in root_list: 
#     if (root.isalpha() or root.isspace()) == True: 
#         new_root_list.append(root)
# #print(len(new_root_list))

# root_list = new_root_list

In [6]:
def GetMuValueFromTree(root, mu_dict):
        if root:
            GetMuValueFromTree(root.prev, mu_dict)
            mu_dict[root.word] = root.mu
            GetMuValueFromTree(root.next, mu_dict)

def FindPhraseInCandidate(phrase, candidate_phrases):
        if len(phrase) > 0:
            for i in range (0, len(candidate_phrases)):
                if phrase == candidate_phrases[i]:
                    return i
            return -1
        else:
            return -1

## Get Final Key Phrases

In [7]:
my_final_list = []
weights = []
my_final_dict = {}

mu = -30

for i in range(len(root_word_list)):
    # print("i = " + str(i))
    root_word = root_word_list[i]
    similar_candidate, similar_TFIDF, root_tfidf = preparation(root = root_word, final_output = final_output, candidate = candidate)

    rootphrase = similar_candidate[0].split()
    rootIndex = find_index(root_word, rootphrase)

    my_tree = Tree()
    my_tree.AddNode(word = root_word, phrase = rootphrase, wordIndex = rootIndex, rootIndex = rootIndex, tfidf = root_tfidf) # Add the root first

    for j in range(len(similar_candidate)):
        phrase = similar_candidate[j].split()
        rootIndex = find_index(root_word, phrase)   # need to find rootIndex in each phrase
        # print("j = " + str(j))
        for wordIndex in range(len(phrase)):
            word = phrase[wordIndex]                    # extract the first word from the phrase with wordIndex
            word_tfidf = similar_TFIDF[j][wordIndex]    # selecting the first sentence
            
            my_tree.AddNode(word, phrase, wordIndex, rootIndex, word_tfidf) 
            # my_tree.PrintTree(my_tree.root)
            # print(str(wordIndex) + "th iteration completed...")

        my_tree.UpdateMuValues(phrase)
        
    # my_tree.PrintTree(my_tree.root)
    # my_NodeList = my_tree.CreateNodeList(my_tree.root)
    # print(my_NodeList)

    # Getting the final phrases list
    final_node_list = []
    final_node_list = my_tree.FindNodeListToExtractKeyPhrases(mu = mu, final_node_list = final_node_list, candidate_phrases= candidate) # This includes tree pruning too
    final_phrases = []
    for j in range(0, len(final_node_list)):
        final_phrases.append(my_tree.GetPhrase(final_node_list[j]))
    # my_final_list.append(final_phrases)


    ## Getting the final mu list

    # Get the mu value for each word
    mu_dict = {} # a dictionary that saves "word : mu" pairs 
    GetMuValueFromTree(my_tree.root, mu_dict)

    # Save the final keyphrases and their weights
    for l in range(0, len(final_phrases)):
        k = FindPhraseInCandidate(final_phrases[l], similar_candidate)
        if  k == -1: 
            continue
        else:
            my_final_list.append(final_phrases[l])

            # calculate the sum of tfidf of all the words in the final keyphrase
            tfidf_value = np.sum(similar_TFIDF[k])
            # calculate the sum of mu of all the words in the final keyphrase
            mu_value = 0 
            for j in final_phrases[l].split():
                mu_value = mu_value + mu_dict[j]
            weights.append(tfidf_value * mu_value)
    
    for m in range(0, len(my_final_list)): 
        my_final_dict[my_final_list[m]] = weights[m]
    print(str(i+1) + "th iteration completed...")

1th iteration completed...
2th iteration completed...
3th iteration completed...
4th iteration completed...
5th iteration completed...
6th iteration completed...
7th iteration completed...
8th iteration completed...
9th iteration completed...
10th iteration completed...
11th iteration completed...
12th iteration completed...
13th iteration completed...
14th iteration completed...
15th iteration completed...
16th iteration completed...
17th iteration completed...
18th iteration completed...
19th iteration completed...
20th iteration completed...
21th iteration completed...
22th iteration completed...
23th iteration completed...
24th iteration completed...
25th iteration completed...
26th iteration completed...
27th iteration completed...
28th iteration completed...
29th iteration completed...
30th iteration completed...
31th iteration completed...
32th iteration completed...
33th iteration completed...
34th iteration completed...
35th iteration completed...
36th iteration completed...
3

In [8]:
my_final_dict_sorted = dict(sorted(my_final_dict.items(), key=lambda item: item[1], reverse = True))
my_final_dict_sorted

{'svm': 173.0,
 'spam': 170.0,
 'onlin': 110.0,
 'data mine': 107.03513161388946,
 'data': 97.0,
 'set': 74.0,
 'comput cost': 73.51550573878438,
 'perform': 73.0,
 'spam comment': 70.72420572990819,
 'rosvm test': 70.57540843056644,
 'support vector machin': 69.0824633143468,
 'result': 69.0,
 'detect': 59.0,
 'exampl xi': 58.65741414907701,
 'method trade': 57.844300807508,
 'margin maxim': 54.02335043906604,
 'exampl': 54.0,
 'vector machin': 53.685064694791016,
 'method': 52.0,
 'valu': 51.0,
 'email': 51.0,
 'benchmark data set': 49.531327081792206,
 'blog': 49.0,
 'cost': 49.0,
 'rosvm': 48.0,
 'splog detect': 43.779988085186915,
 'vector': 42.0,
 'contentbas detect': 39.82670598659872,
 'word vector': 39.53729646210428,
 'number': 39.0,
 'margin': 39.0,
 'spam filter': 38.639081951240364,
 'comput': 38.0,
 'featur': 36.0,
 'method trade statist robust': 35.445632473717154,
 'splog': 35.0,
 'support vector learn': 34.44944253419359,
 'blog identif': 34.283856990890094,
 'time': 3

In [9]:
# ind = 0
# cnt = 0
# for c in range(len(candidate)):
#     sent = candidate[c]
#     if sent == 'resourc':
#         cnt = cnt + 1
#         # print(ind)
#     ind = ind + 1
# print(cnt)

# candidate[255]

In [13]:
# key = list(my_final_dict_sorted.keys())
# value = list(my_final_dict_sorted.values())

# final_real_phrases = {}
# for i in range(len(key)):
#     if len(key[i].split()) > 1:
#         final_real_phrases[key[i]] = value[i]
# final_real_phrases

In [14]:
final_real_phrases = my_final_dict_sorted

N = 20
N_phrases = list(final_real_phrases.keys())[0:N]
print("(1) Using ExtractCandidate()")
N_phrases

(1) Using ExtractCandidate()


['svm',
 'spam',
 'onlin',
 'data mine',
 'data',
 'set',
 'comput cost',
 'perform',
 'spam comment',
 'rosvm test',
 'support vector machin',
 'result',
 'detect',
 'exampl xi',
 'method trade',
 'margin maxim',
 'exampl',
 'vector machin',
 'method',
 'valu']

In [16]:
standard = ['support vector machin', 'content-base filter', 'spam filter', 'blog', 'splog', 'link analysi', 
'machin learn techniqu', 'link spam', 'content-base spam detect', 'bayesian method', 'increment updat', 
'logist regress', 'hyperplan', 'featur map', 'spam filter']

1

In [17]:
K_correct = 0
for word in N_phrases:
    if word in standard:
        K_correct = K_correct + 1
K_correct

1

#### Gold Standard (Stem version)

[author]
distribut real-time emb system,hybrid system,qualiti of servic+servic qualiti

[reader]
adapt resourc manag,distribut real-time embed system,end-to-end qualiti of servic+servic end-to-end qualiti,hybrid adapt resourcemanag middlewar,hybrid control techniqu,real-time video distribut system,real-time corba specif,video encod/decod,resourc reserv mechan,dynam environ,stream servic


[Combined]
adapt resourc manag,distribut real-time embed system,end-to-end qualiti of servic+servic end-to-end qualiti,hybrid adapt resourcemanag middlewar,hybrid control techniqu,real-time video distribut system,real-time corba specif,video encod/decod,resourc reserv mechan,dynam environ,stream servic,distribut real-time emb system,hybrid system,qualiti of servic+servic qualiti